## Real Example: Logistic Regression

Remembering the model: given $X \subset R^n$ and $Y \in \{0, 1\}$ we want to model $P(Y | X)$

We adjust variables ("weights" $w \in R^n$ e $b \in R$ in a function $\phi(w^T x + b)$, with $\phi$ being the logistic function: $\phi(x) = \frac{1}{1 + e^{-x}}$ in order to approximate our function to the data distribution

We calculate the distance from our function to the data's by means of an error function, such as cross-entropy

Logistic regression has no closed formula such as linear regression, so it has to be approximated iteratively with, for example, gradient descent, something very close to what we'll do later with neural networks.


In [3]:
import numpy as np
import theano
import theano.tensor as T
rng = np.random

#We generate a random dataset
N     = 400  # number of samples
feats = 80  # dimension of X

reg = 0.01 
lrate = 0.1
train_steps = 10000

D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))

In [4]:
#We declare the variables of our model
w = theano.shared(rng.randn(feats), name='w')
b = theano.shared(0., name='b')

print("Initial model:")
print(w.get_value())
print(b.get_value())

Initial model:
[ 0.10800069 -0.07279783 -0.72164074 -0.66693937 -0.61069386 -1.78820542
  2.16634618 -1.59271677  1.36084087 -0.53061822  0.30847367 -0.40538653
 -0.85365458 -0.48251394  0.88621032 -2.59412079  0.52250828  1.52773399
  0.08452786 -0.97482446 -1.38965265 -1.09351247 -0.76136163 -0.58415
  0.52483069 -1.83065274 -0.2129953  -0.63574292  1.35014302 -0.27030217
  0.12460241 -1.94232726  0.08663744 -0.39646824 -0.68007583  0.56808543
 -1.31082638  0.22092426  0.55359826  0.28123096 -0.51917937 -0.52217558
  0.53099103 -0.11108518 -0.80694072  0.28949837 -1.30350523 -0.75745774
 -0.22662349 -0.10987074  0.78299616  1.18349776  0.79349253 -1.39635317
  1.29984518  0.58251052  0.44642803 -1.43446101 -1.99874319 -1.44178027
 -0.84214434  0.24348001  1.29897971 -0.01573493 -0.8485208   0.20546356
  0.38350427  0.13353544  0.14411761 -0.2131891  -0.76707768  0.95729936
  0.57249133 -0.39531144  0.16787614 -0.18501385  2.1389884   0.4533837
 -0.23738353  0.28990437]
0.0


In [5]:
#We declare the entry points and construct the graph
x = T.dmatrix('x')
y = T.dvector('y')

prob = 1 / (1 + T.exp(-T.dot(x, w) - b)) #logistic function 
xent = -y * T.log(prob) - (1-y) * T.log(1-prob)  #cross-entropy as error function
cost = xent.mean() + reg * (w ** 2).sum() #cost function with added regularization

#gradients we'll need for updates
gw = T.grad(cost, w) 
gb = T.grad(cost, b)

pred = prob > 0.5 #final classification

train = theano.function(
            inputs =[x,y],
            outputs=[cost],
            updates =((w, w - lrate * gw), (b, b - lrate * gb))
)
predict = theano.function(inputs=[x], outputs=pred)

In [6]:
#Training
for i in range(train_steps):
    err = train(D[0], D[1])
    
#Verification
score = np.mean(predict(D[0]) == D[1])
print(score)

#We take a look at our weights and bias if we are interested in them
print(w.get_value())
print(b.get_value())

0.675
[-0.05585622  0.03631046 -0.13132262 -0.20585999 -0.07649237  0.10989294
 -0.01910302  0.04738287  0.04093607  0.02550771  0.32558103  0.12717924
 -0.13923911 -0.05022331  0.16277569  0.11356429  0.01796868  0.11192106
  0.02748607 -0.16713429 -0.0351308  -0.10702765 -0.25738587 -0.00302086
 -0.06912819  0.11896982 -0.01053852 -0.08332012  0.06024302 -0.09008435
 -0.03907883  0.14187424  0.00689776 -0.01418794  0.13285188 -0.06001423
  0.36658847  0.12595354 -0.10419609 -0.09019202 -0.16989732  0.2430947
  0.18654872  0.08817617 -0.04825535  0.11083986  0.17506654 -0.0905112
 -0.07661157  0.02270848 -0.04991576 -0.1505727  -0.17620851 -0.09534404
 -0.04322099 -0.12926032  0.00538425 -0.20306491  0.14788061  0.11208898
  0.03388175 -0.08326146 -0.12161601 -0.04338878  0.14528264  0.10174629
 -0.08984361 -0.07766124  0.20091132 -0.07662668 -0.10604438 -0.02502383
 -0.03576966  0.01543179 -0.14433605 -0.09883313 -0.04952134 -0.10804917
  0.09427666  0.08131202]
0.09623414973420996
